## Imports

In [1]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
from sklearn.metrics import classification_report
import warnings 
import numpy as np
warnings.filterwarnings("ignore")
import seaborn as sns

In [2]:
from math import isnan

In [3]:
import sys
sys.path.append('../src')
from model.proba_model import get_past_meta, predict
from model.evaluation import get_classification_report

## Definitions

In [7]:
# ### Definitions
path_data_dir = '../data/'

list_assets = ["PETR3.SA","PRIO3.SA", "VALE3.SA", "GGBR3.SA", "ABCB4.SA", "ITUB3.SA", "FLRY3.SA", "RADL3.SA"]

relevant_cols = ['Date', 'Close', 'Volume']

windows = [7,14,21]

In [8]:
list_prev_meta = [i for i in range(1,6)]

## Classification

### Dummy Model

#### Temporal train test split

In [ ]:
n_prev_meta = 1


list_results = []




for asset in list_assets:
    
    print(f"""
#################
# asset: {asset}
#################          
          """)
    
    # get train contigency table 
    cont_tbl_train = pd.read_csv(path_data_dir + f"processed/train_contingency_table_price_history_{asset.replace('.', '_')}_meta_range({n_prev_meta})_dataset_ffill.csv", index_col=0, header=[i for i in range(n_prev_meta)])
    
    # create probability table
    df_probas = cont_tbl_train.apply(lambda x: [col/sum(x) for col in x])    
    
    # get test dataset
    test_dataset = pd.read_csv(path_data_dir + f"processed/test_price_history_{asset.replace('.', '_')}_meta_dataset_ffill.csv", index_col= 0)
    
    # get the past meta for all days    
    test_dataset["past_meta"] = test_dataset.apply(lambda x: get_past_meta(test_dataset,x,n_prev_meta,name_meta_col = "meta"), axis=1)

    # remove the first rows
    test_dataset = test_dataset.iloc[n_prev_meta:]
    
    # get y_test
    y_test = test_dataset.meta
    
    # predict based on probability table
    y_pred = test_dataset.past_meta.astype('int')
    
    # create df with results
    df_results_p = get_classification_report(y_test, y_pred)
    df_results_p['asset'] = asset
    
    print(classification_report(y_test, y_pred))
    
    
    list_results.append(df_results_p)
    
    df_results_p = df_results_p.drop(['weighted avg', 'macro avg', 'accuracy'])
    df_results_p.index = df_results_p.index.astype('int')
    df_results_p = df_results_p.reset_index()
    df_results_p.support = df_results_p.support.astype('int')
    
    df_results_p.drop(['asset'],axis = 1,inplace = True)
    df_results_p.rename(
        {
            'index':'p',
            "precision": 'precision',
            "recall": 'recall',
            "f1-score": 'f1-score',
            "support": "q"
        }
        
        
        ,axis = 1, inplace = True)

    df_results_p.sort_values('p', inplace = True)
    print(df_results_p.to_latex(index=False,float_format="%.2f"))
    
    
    
    
df_results = pd.concat(list_results)


#################
# asset: PETR3.SA
#################          
          


TypeError: unsupported operand type(s) for -: 'str' and 'int'